In [1]:
!pip install gradio
import gradio as gr
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import datetime

def predict_stock(stock_symbol, start_date, end_date, forecast_horizon):
    # Convert date strings to datetime objects
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()

    # Step 1: Fetch historical stock data
    df = yf.download(stock_symbol, start=start_date, end=end_date)
    if df.empty:
        return "Error: No data found for this symbol and date range.", None

    df = df['Close'].fillna(method='ffill')

    # Step 2: Preprocess (already filled missing; could normalize here)
    series = df.values
    model = ARIMA(series, order=(5, 1, 0))  # Dummy ARIMA config
    model_fit = model.fit()

    # Step 3: Forecast
    forecast = model_fit.forecast(steps=int(forecast_horizon))

    # Step 4: Visualization
    fig, ax = plt.subplots()
    ax.plot(df.index, df.values, label='Historical')
    future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=int(forecast_horizon), freq='B')
    ax.plot(future_dates, forecast, label='Forecast', color='red')
    ax.set_title(f"{stock_symbol} Stock Price Forecast")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Step 5: Dummy interpretability (SHAP placeholder)
    interpretation = f"The prediction is mostly influenced by recent price drops and volatility before {df.index[-1].strftime('%Y-%m-%d')}."

    return interpretation, fig

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 📈 Real-Time Stock Market Predictor with Gradio")
    gr.Markdown("Enter stock parameters below to predict future prices using ARIMA (replaceable with LSTM/SARIMA).")

    with gr.Row():
        stock_symbol = gr.Textbox(label="Stock Symbol (e.g., AAPL)", value="AAPL")
        forecast_horizon = gr.Slider(5, 30, step=1, label="Forecast Horizon (Days)", value=10)

    with gr.Row():
        start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)", value=(datetime.date.today() - datetime.timedelta(days=180)).strftime('%Y-%m-%d'))
        end_date = gr.Textbox(label="End Date (YYYY-MM-DD)", value=datetime.date.today().strftime('%Y-%m-%d'))

    predict_btn = gr.Button("Predict")

    output_text = gr.Textbox(label="Interpretation")
    output_plot = gr.Plot(label="Forecasted Stock Price")

    predict_btn.click(fn=predict_stock,
                      inputs=[stock_symbol, start_date, end_date, forecast_horizon],
                      outputs=[output_text, output_plot])

demo.launch()

/usr/local/lib/python3.11/dist-packages/gradio/analytics.py:108: UserWarning: unable to parse version details from package URL.
  warnings.warn("unable to parse version details from package URL.")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b9841428107cfbed10.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
